Setting up a NN Workflow
======

This is where I am taking the puzzle pieces I learned from `deep_learning_keras.ipynb` and applying them to make
moves towards deploying NN in our code base.  I am using a notebook because I still want the visuals for explaining
the process and seeing the outputs.  Once the code is ironed out, we will move it to the `core` codebase.
***
## Objectives
1. Import data and featurize with RDKit and/or ECFP
1. Create functions for defining basic neural networks with minimum manual repitition.
2. Set up procedures to optimize the model using Bayesian methods for hyperparamters.
3. Implement callbacks for saving the model and rolling back to the best version.
5. Visualize the results of the model using `matplotlib` and Tensorboard.
6. Export stats and results similarly to how we have done it in the past.

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Function to define neural network
Using the Sequential API to make architectures with square or pyramid shape.  Pyramids will be wide at the input and narrow at the
output.

How can I handle the variable input shapes?  You really need to define it, I think. If I make it a requirement,
how does that affect wrapping it up in sklearn?

In [2]:
def build_nn(n_hidden = 2, n_neuron = 50, learning_rate = 1e-3, in_shape=[200]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=in_shape))  # input layer.  How to handle shape?
    for layer in range(n_hidden):  # create hidden layers
        model.add(keras.layers.Dense(n_neuron, activation="relu"))
    model.add(keras.layers.Dense(1))  # output layer
    optimizer = keras.optimizers.SGD(lr=learning_rate)  # this is a point to vary.  Dict could help call other ones.
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    # optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    return model

Some thoughts on the above function.  I could add an option for specifying the overall shape, for example `"rect"` or
`"triangle"`. This would impact how we taper the number of neurons as the layers get deeper.

It would be nice to be able to play with the activation function from outside the function.  Could be dictionary use.
Similarly, it would be nice to be able to optimize the optimizer but retain the granular tuning provided by calling
the specific `keras.optimizers`.  Again, this could be a nice place for calling a function from dictionary.
One challenge with this would be passing parameters to it.  I think most will have a learning rate parameter, but other
than that, how would we automatically tune such parameters?  That may be too advanced for this scope of work.

### Import and process data
Use existing processes to featurize and split some data for use with the models.



In [3]:
# from core.ingest import load_smiles
from features import targets_features, featurize
# from core.misc import cd

Load the data using our `ingest.py`

In [4]:
# specific to 18k-logP
data = {'18k-logP.csv':'logp'}
dataset = '18k-logP.csv'
#
# with cd('../dataFiles/'): # move to dataset directory
#     df, exp = load_smiles(dataset, data[dataset])
#
# df.head(5)

Featurize the dataset using our `features.featurize()`.  I don't think it really matters at this point what algorithm
name gets passed.  I will handle the normalization/scaling manually.

In [5]:
# df, num_feat, feat_time = featurize(df, 'rf', [0])
df = pd.read_csv('logP-18k_feat.csv', index_col=0)
df.head(5)

,smiles,logp,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,qed
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54,1.420544,832.199002,16.518297,13.821155,14.577084,11.703510,8.337640,8.715604,...,0,0,0,0,0,0,0,0,0,0.728444
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18,2.020016,1151.428500,24.172998,18.530293,20.163286,15.683108,10.164097,12.758861,...,1,0,0,0,0,0,0,0,0,0.545587
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69,1.943224,655.231463,14.819626,11.712695,13.285121,10.202709,6.819775,8.077393,...,0,0,0,0,0,0,1,0,0,0.807761
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37,1.572408,1015.409752,19.836134,14.684473,16.256898,13.456729,8.731046,9.925507,...,0,0,0,0,0,0,1,0,0,0.506650
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10,2.236144,902.250256,20.896977,17.036453,17.036453,13.112411,9.171258,9.171258,...,0,0,0,0,0,0,0,0,0,0.747686


In [6]:
# df = df.drop("RDKit2D_calculated", axis=1)
# df.to_csv('logP-18k_feat.csv')
# df.head(5)

Split the data.  I am unsure if we need a specific validation set because the goal is to use cross validation (CV),
which does not require validation data to be set aside.  However, I will leave the validation set as is for
trouble-shooting purposes for now.

In [7]:
# split up the data using 20% for testing
train_features_full, test_features, train_target_full, test_target, feature_list = targets_features(df, data[dataset])

# get validation data from training data (20% of 80%? )
train_features, val_features, train_target, val_target = train_test_split(train_features_full,train_target_full)

Scale data using sklearn `StandardScalar()`

In [8]:
# scale feature vectors
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

In [9]:
df.shape
n_total = df.shape[0]
n_total

18362

In [10]:
n_train = train_features.shape[0]
ptrain = n_train / n_total *100
n_train

11016

In [11]:
n_test = test_features.shape[0]
ptest = n_test / n_total *100
n_test

3673

In [12]:
n_val = val_features.shape[0]
pval = n_val / n_total *100
pval
# n_val

20.003267617906545

In [13]:
n_train + n_val+ n_test == n_total
print('The dataset of {} points is split into training ({:.1f}%), validation ({:.1f}%), and testing ({:.1f}%).'.format(n_total, ptrain, pval, ptest ))


The dataset of 18362 points is split into training (60.0%), validation (20.0%), and testing (20.0%).


### Deploy the model and data

The input shape is the part that I find a bit tricky, but lets see how it goes.  So since the the input shape is
defined as a parameter to the initial model, it seems like I need to treat it like a hyper parameter.  One way to do
this is to include it in the hyper parameter dictionary, but that is meant to be static and the input shape will not be
static.  I could append in a shape to the dict at run time and what I append could be dynamic.

### Create the callbacks for saving and early stoppage

In [14]:
model = build_nn(n_hidden=50, n_neuron=300, in_shape=train_features.shape[1:], learning_rate=0.001)

# set a checkpoint file to save the model
chkpt_cb = keras.callbacks.ModelCheckpoint('bayes_logP.h5', save_best_only=True)
# set up early stopping callback to avoid wasted resources
stop_cb = keras.callbacks.EarlyStopping(patience=10,  # number of epochs to wait for progress
                                        restore_best_weights=True)

# print(callable(keras.callbacks.EarlyStopping) 

### Create the scikit learn wrapper

In [15]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_nn, in_shape=200)  # pass non-hyper params here

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal
import numpy as np
neurons = np.arange(20,100)
print(neurons)
param_dict = {
    "n_hidden": [1,2,3,4],
    "n_neuron": neurons,
    "learning_rate": np.logspace(-4, 0, num=5, base=11)
}

print(np.arange(20,100))
print(param_dict)
# data_shape = {"in_shape": train_features.shape[1:]}
# print(data_shape)
# # param_dict["in_shape"] = train_features.shape[1:]  # add input shape to parameter dictionary is one way to fix
# print(param_dict)
# # print(param_dict.update(data_shape))
print("new")
print(np.logspace(-4, 0, num=11, base=10))

[20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67
 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91
 92 93 94 95 96 97 98 99]
[20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67
 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91
 92 93 94 95 96 97 98 99]
{'n_hidden': [1, 2, 3, 4], 'n_neuron': array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
       71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
       88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'learning_rate': array([6.83013455e-05, 7.51314801e-04, 8.26446281e-03, 9.09090909e-02,
       1.00000000e+00])}
n

### Bayesian Optimization
Smarter optmization, but you must define parameter grids in a slightly different way than base sklearn.
Specifically, you must include the type of parameter it is.

In [ ]:
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV

bayes_grid = {
        'n_hidden': Integer(1, 10),
        'n_neuron': Integer(50,300),
        'learning_rate': Real(0.0001, 0.1, 'log-uniform')
}
print('ready to go')
print(Real(0.001, 1, 'log-uniform'))

In [ ]:
def get_params(self, **_params): return dict(self.sk_params, build_fn=self.build_fn)

In [ ]:
# set up Bayes Search
bayes = BayesSearchCV(
    estimator=keras_reg,  # what regressor to use
    search_spaces=bayes_grid,  # hyper parameters to search through
    n_iter=25,  # number of combos tried
    random_state=42,  # random seed
    verbose=3,  # output print level
    fit_params = {'epochs': 50, 'callbacks': [chkpt_cb, stop_cb], 'validation_data': (val_features,val_target)},
    scoring='neg_mean_squared_error',  # scoring function to use (RMSE)
    # n_jobs=1,  # number of parallel jobs (max = folds)
    cv=10  # number of cross-val folds to use
)
# bayes. get_params

# Fit the Bayes search model
bayes.fit(train_features, train_target)



In [ ]:
tuned = bayes.best_params_
tune_score = bayes.best_score_
print("Bayes Best Params:", tuned)
print("Bayes Best Score:", tune_score)

best_model = bayes.best_estimator_.model
mse_test_opt = best_model.evaluate(test_features, test_target)
print('\n Bayes Test MSE {:.4f}'.format(mse_test_opt[0]))
print('\n Bayes Test RMSE {:.4f}'.format(mse_test_opt[1]))

In [ ]:
# random_search = BayesSearchCV(estimator=keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_nn, in_shape=200,
#                                                             epochs=10,
#                                                             verbose=2
#                                                            ),
#                             search_spaces=bayes_grid,  # hyper parameters to search through
#                             n_iter=5,  # number of combos tried
#                             random_state=42,  # random seed
#                             verbose=3,  # output print level
#                             scoring='neg_mean_squared_error',  # scoring function to use (RMSE)
#                             # n_jobs=1,  # number of parallel jobs (max = folds)
#                             cv=3  # number of cross-val folds to use
# )
# random_search.fit(train_features, train_target,
# #                     validation_data=(val_features,val_target),
#                     callback=keras.callbacks.EarlyStopping(patience=10,  # number of epochs to wait for progress
#                                         restore_best_weights=True))

In [ ]:
# rnd_search = RandomizedSearchCV(keras_reg, param_dict, n_iter=2, cv=3, verbose=3, n_jobs = 1)
# rnd_search.fit(train_features, train_target, epochs=50,
#                     validation_data=(val_features,val_target),
#                     callbacks=[chkpt_cb, stop_cb])

In [ ]:
# best_param = rnd_search.best_params_
# print("Random Best Params:", best_param)
# best_model = rnd_search.best_estimator_.model
# mse_test_opt = best_model.evaluate(test_features, test_target)
# print('\nRandom Test MSE {:.4f}'.format(mse_test_opt))

In [16]:
# history = model.fit(train_features, train_target, epochs=50,
#                     validation_data=(val_features,val_target),
#                     callbacks=[chkpt_cb, stop_cb])
#
# mse_test = model.evaluate(test_features, test_target)
# print('\nTest MSE {:.4f}'.format(mse_test))

ready to go
Real(low=0.001, high=1, prior='log-uniform', transform='identity')


C:\Users\Adam\miniconda3\envs\mlapp\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [17]:
def get_params(self, **_params): return dict(self.sk_params, build_fn=self.build_fn)

In [ ]:
# set up Bayes Search
bayes = BayesSearchCV(
    estimator=keras_reg,  # what regressor to use
    search_spaces=bayes_grid,  # hyper parameters to search through
    n_iter=25,  # number of combos tried
    random_state=42,  # random seed
    verbose=3,  # output print level
    fit_params = {'epochs': 50, 'callbacks': [chkpt_cb, stop_cb], 'validation_data': (val_features,val_target)},
    scoring='neg_mean_squared_error',  # scoring function to use (RMSE)
    # n_jobs=1,  # number of parallel jobs (max = folds)
    cv=10  # number of cross-val folds to use
)
# bayes. get_params

# Fit the Bayes search model
bayes.fit(train_features, train_target)



Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] learning_rate=0.0016994636371262762, n_hidden=8, n_neuron=283 ...
Train on 9914 samples, validate on 3673 samples
Epoch 1/50


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


9914/9914 [==============================] - 2s 201us/sample - loss: 1.1900 - rmse: 1.0909 - val_loss: 0.6487 - val_rmse: 0.8054
Epoch 2/50
9914/9914 [==============================] - 1s 149us/sample - loss: 0.5865 - rmse: 0.7658 - val_loss: 0.5879 - val_rmse: 0.7668
Epoch 3/50
9914/9914 [==============================] - 1s 135us/sample - loss: 0.4641 - rmse: 0.6812 - val_loss: 0.6353 - val_rmse: 0.7970
Epoch 4/50
9914/9914 [==============================] - 1s 143us/sample - loss: 0.4233 - rmse: 0.6506 - val_loss: 0.4776 - val_rmse: 0.6911
Epoch 5/50
9914/9914 [==============================] - 1s 138us/sample - loss: 0.4017 - rmse: 0.6338 - val_loss: 0.4422 - val_rmse: 0.6650
Epoch 6/50
9914/9914 [==============================] - 1s 148us/sample - loss: 0.3350 - rmse: 0.5788 - val_loss: 0.4966 - val_rmse: 0.7047
Epoch 7/50
9914/9914 [==============================] - 1s 133us/sample - loss: 0.3177 - rmse: 0.5637 - val_loss: 0.4455 - val_rmse: 0.6675
Epoch 8/50
9914/9914 [=========

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


9914/9914 [==============================] - 2s 207us/sample - loss: 1.0200 - rmse: 1.0100 - val_loss: 0.6790 - val_rmse: 0.8240
Epoch 2/50
9914/9914 [==============================] - 2s 158us/sample - loss: 0.6178 - rmse: 0.7860 - val_loss: 0.5486 - val_rmse: 0.7407
Epoch 3/50
9914/9914 [==============================] - 2s 159us/sample - loss: 0.4881 - rmse: 0.6987 - val_loss: 0.4894 - val_rmse: 0.6996
Epoch 4/50
9914/9914 [==============================] - 1s 151us/sample - loss: 0.4300 - rmse: 0.6557 - val_loss: 0.5626 - val_rmse: 0.7501
Epoch 5/50
9914/9914 [==============================] - 1s 135us/sample - loss: 0.4014 - rmse: 0.6336 - val_loss: 0.5008 - val_rmse: 0.7077
Epoch 6/50
9914/9914 [==============================] - 1s 148us/sample - loss: 0.3582 - rmse: 0.5985 - val_loss: 0.5266 - val_rmse: 0.7257
Epoch 7/50
9914/9914 [==============================] - 2s 153us/sample - loss: 0.3227 - rmse: 0.5681 - val_loss: 0.4404 - val_rmse: 0.6636
Epoch 8/50
9914/9914 [=========

...

9914/9914 [==============================] - 2s 227us/sample - loss: 876820.4417 - rmse: 936.3848 - val_loss: 2.9242 - val_rmse: 1.7100
Epoch 2/50
9914/9914 [==============================] - 1s 143us/sample - loss: 3.0073 - rmse: 1.7341 - val_loss: 2.9241 - val_rmse: 1.7100
Epoch 3/50
9914/9914 [==============================] - 2s 152us/sample - loss: 2.9811 - rmse: 1.7266 - val_loss: 2.9249 - val_rmse: 1.7102
Epoch 4/50
9914/9914 [==============================] - 2s 160us/sample - loss: 2.9667 - rmse: 1.7224 - val_loss: 2.9301 - val_rmse: 1.7118
Epoch 5/50
9914/9914 [==============================] - 1s 150us/sample - loss: 2.9773 - rmse: 1.7255 - val_loss: 3.0188 - val_rmse: 1.7375
Epoch 6/50
9914/9914 [==============================] - 2s 160us/sample - loss: 2.9952 - rmse: 1.7307 - val_loss: 2.9270 - val_rmse: 1.7109
Epoch 7/50
9914/9914 [==============================] - 2s 172us/sample - loss: 2.9976 - rmse: 1.7314 - val_loss: 2.9446 - val_rmse: 1.7160
Epoch 8/50
9914/9914 [==

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.3s remaining:    0.0s


9914/9914 [==============================] - 2s 220us/sample - loss: 9158849.8178 - rmse: 3026.3586 - val_loss: 2.9244 - val_rmse: 1.7101
Epoch 2/50
9914/9914 [==============================] - 2s 163us/sample - loss: 2.9748 - rmse: 1.7248 - val_loss: 2.9256 - val_rmse: 1.7104
Epoch 3/50
9914/9914 [==============================] - 2s 156us/sample - loss: 3.0052 - rmse: 1.7336 - val_loss: 2.9248 - val_rmse: 1.7102
Epoch 4/50
9914/9914 [==============================] - 2s 158us/sample - loss: 2.9844 - rmse: 1.7275 - val_loss: 2.9301 - val_rmse: 1.7117
Epoch 5/50
9914/9914 [==============================] - 1s 150us/sample - loss: 2.9808 - rmse: 1.7265 - val_loss: 2.9706 - val_rmse: 1.7235
Epoch 6/50
9914/9914 [==============================] - 2s 155us/sample - loss: 2.9876 - rmse: 1.7285 - val_loss: 2.9652 - val_rmse: 1.7220
Epoch 7/50
9914/9914 [==============================] - 1s 146us/sample - loss: 2.9757 - rmse: 1.7250 - val_loss: 2.9266 - val_rmse: 1.7107
Epoch 8/50
9914/9914 [

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   36.8s remaining:    0.0s



Epoch 1/50
9914/9914 [==============================] - 2s 222us/sample - loss: 6841635.0284 - rmse: 2615.6514 - val_loss: 2.9856 - val_rmse: 1.7279
Epoch 2/50
9914/9914 [==============================] - 1s 143us/sample - loss: 8553549.1190 - rmse: 2924.6450 - val_loss: 39.6529 - val_rmse: 6.2971
Epoch 3/50
9914/9914 [==============================] - 1s 140us/sample - loss: 549269.6537 - rmse: 741.1259 - val_loss: 2.9241 - val_rmse: 1.7100
Epoch 4/50
9914/9914 [==============================] - 1s 149us/sample - loss: 2.9635 - rmse: 1.7215 - val_loss: 3.1006 - val_rmse: 1.7609
Epoch 5/50
9914/9914 [==============================] - 1s 149us/sample - loss: 2.9877 - rmse: 1.7285 - val_loss: 3.3057 - val_rmse: 1.8182
Epoch 6/50
9914/9914 [==============================] - 1s 145us/sample - loss: 3.0330 - rmse: 1.7416 - val_loss: 2.9323 - val_rmse: 1.7124
Epoch 7/50
9914/9914 [==============================] - 1s 138us/sample - loss: 3.0347 - rmse: 1.7420 - val_loss: 3.2507 - val_rmse: 

In [ ]:
tuned = bayes.best_params_
tune_score = bayes.best_score_
print("Bayes Best Params:", tuned)
print("Bayes Best Score:", tune_score)

best_model = bayes.best_estimator_.model
mse_test_opt = best_model.evaluate(test_features, test_target)
print('\n Bayes Test MSE {:.4f}'.format(mse_test_opt[0]))
print('\n Bayes Test RMSE {:.4f}'.format(mse_test_opt[1]))

In [ ]:
# random_search = BayesSearchCV(estimator=keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_nn, in_shape=200,
#                                                             epochs=10,
#                                                             verbose=2
#                                                            ),
#                             search_spaces=bayes_grid,  # hyper parameters to search through
#                             n_iter=5,  # number of combos tried
#                             random_state=42,  # random seed
#                             verbose=3,  # output print level
#                             scoring='neg_mean_squared_error',  # scoring function to use (RMSE)
#                             # n_jobs=1,  # number of parallel jobs (max = folds)
#                             cv=3  # number of cross-val folds to use
# )
# random_search.fit(train_features, train_target,
# #                     validation_data=(val_features,val_target),
#                     callback=keras.callbacks.EarlyStopping(patience=10,  # number of epochs to wait for progress
#                                         restore_best_weights=True))

In [ ]:
# rnd_search = RandomizedSearchCV(keras_reg, param_dict, n_iter=2, cv=3, verbose=3, n_jobs = 1)
# rnd_search.fit(train_features, train_target, epochs=50,
#                     validation_data=(val_features,val_target),
#                     callbacks=[chkpt_cb, stop_cb])

In [ ]:
# best_param = rnd_search.best_params_
# print("Random Best Params:", best_param)
# best_model = rnd_search.best_estimator_.model
# mse_test_opt = best_model.evaluate(test_features, test_target)
# print('\nRandom Test MSE {:.4f}'.format(mse_test_opt))

In [ ]:
# history = model.fit(train_features, train_target, epochs=50,
#                     validation_data=(val_features,val_target),
#                     callbacks=[chkpt_cb, stop_cb])
#
# mse_test = model.evaluate(test_features, test_target)
# print('\nTest MSE {:.4f}'.format(mse_test))

In [ ]:
import sklearn 
sklearn.__version__